In [ ]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Conv1D, GlobalMaxPooling1D, Dropout, LSTM, GRU
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
!pip install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset


In [ ]:
dataset = load_dataset('ag_news')
train_dataset = dataset['train']
test_dataset = dataset['test']


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import datasets
ag_news = datasets.load_dataset("ag_news")
data = ag_news["train"]
data.to_csv("ag_news.csv", index=False)



  0%|          | 0/2 [00:00<?, ?it/s]

Creating CSV from Arrow format:   0%|          | 0/120 [00:00<?, ?ba/s]

28924927

0 World
1 Sports
2 Business
3 Sci/Tech

In [ ]:
!head ag_news.csv

text,label
"Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.",2
"Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.",2
Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.,2
"Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.",2
"Oil prices soar to all-time record, posing new menace to US economy (AFP

In [ ]:
train = pd.read_csv('ag_news.csv', header=None, names=['text', 'class'], skiprows=1)


In [ ]:
train['class'] = train['class'].astype(str)
train.to_csv('ag_news_cleaned.csv', index=False)

In [ ]:
train = pd.read_csv('ag_news.csv', header=None, names=['text', 'class'])
train = train.iloc[1:]


In [ ]:
train = train[['class', 'text']]

In [ ]:
train

,class,text
1,2,Wall St. Bears Claw Back Into the Black (Reute...
2,2,Carlyle Looks Toward Commercial Aerospace (Reu...
3,2,Oil and Economy Cloud Stocks' Outlook (Reuters...
4,2,Iraq Halts Oil Exports from Main Southern Pipe...
5,2,"Oil prices soar to all-time record, posing new..."
...,...,...
119996,0,Pakistan's Musharraf Says Won't Quit as Army C...
119997,1,Renteria signing a top-shelf deal Red Sox gene...
119998,1,Saban not going to Dolphins yet The Miami Dolp...
119999,1,Today's NFL games PITTSBURGH at NY GIANTS Time...


In [ ]:
news = train['text'].astype(str)
news[:5]

1    Wall St. Bears Claw Back Into the Black (Reute...
2    Carlyle Looks Toward Commercial Aerospace (Reu...
3    Oil and Economy Cloud Stocks' Outlook (Reuters...
4    Iraq Halts Oil Exports from Main Southern Pipe...
5    Oil prices soar to all-time record, posing new...
Name: text, dtype: object

In [ ]:
num_words = 100000
max_news_len = 170
nb_classes = 4

In [ ]:
y_train = utils.to_categorical(train['class'], nb_classes)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(train['class'])
y_train = utils.to_categorical(encoded_labels, nb_classes)

In [ ]:
y_train

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [ ]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(news)

In [ ]:
sequences = tokenizer.texts_to_sequences(news)

In [ ]:
x_train = pad_sequences(sequences, maxlen=max_news_len)
x_train[:5]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
      

In [ ]:
model = Sequential()
model.add(Embedding(num_words, 64, input_length=max_news_len))
model.add(Conv1D(128, 5, padding='valid', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(nb_classes, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.fit(x_train, y_train, epochs=10, batch_size=128, validation_split=0.1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 170, 64)           6400000   
                                                                 
 conv1d (Conv1D)             (None, 166, 128)          41088     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 4)                 260       
                                                        

In [ ]:
model_cnn_save_path = 'modelwhatabout.h5'
checkpoint_callback_cnn = ModelCheckpoint(model_cnn_save_path,
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

In [ ]:
history_cnn = model.fit(x_train,
                            y_train,
                            epochs=5,
                            batch_size=128,
                            validation_split=0.1,
                            callbacks=[checkpoint_callback_cnn])

Epoch 1/5
844/844 [==============================] - ETA: 0s - loss: 0.0078 - accuracy: 0.9976
Epoch 1: val_accuracy improved from -inf to 0.89225, saving model to modelwhatabout.h5
844/844 [==============================] - 228s 271ms/step - loss: 0.0078 - accuracy: 0.9976 - val_loss: 0.6485 - val_accuracy: 0.8923
Epoch 2/5
844/844 [==============================] - ETA: 0s - loss: 0.0077 - accuracy: 0.9974
Epoch 2: val_accuracy improved from 0.89225 to 0.89450, saving model to modelwhatabout.h5
844/844 [==============================] - 227s 269ms/step - loss: 0.0077 - accuracy: 0.9974 - val_loss: 0.6545 - val_accuracy: 0.8945
Epoch 3/5
844/844 [==============================] - ETA: 0s - loss: 0.0064 - accuracy: 0.9980
Epoch 3: val_accuracy did not improve from 0.89450
844/844 [==============================] - 224s 265ms/step - loss: 0.0064 - accuracy: 0.9980 - val_loss: 0.6853 - val_accuracy: 0.8938
Epoch 4/5
844/844 [==============================] - ETA: 0s - loss: 0.0064 - accu

In [ ]:
model.load_weights(model_cnn_save_path)

In [ ]:
from tensorflow import keras
model = keras.models.load_model('modelwhatabout.h5')

In [ ]:
news_article = "A 38-year-old woman in the US, who was apprehended twice for allegedly trying to jump the White House fence last week, has been arrested for scaling a fence at the Treasury Building next to the White House."

news_article_sequence = tokenizer.texts_to_sequences([news_article])
news_article_sequence = pad_sequences(news_article_sequence, maxlen=max_news_len)
predictions = model.predict(news_article_sequence)
print(predictions)

1/1 [==============================] - 0s 135ms/step
[[6.2625903e-01 8.5539051e-09 3.7374079e-01 1.7311436e-07]]


In [ ]:
class_labels = ['World', 'Sports', 'Business', 'Sci/Tech']
predictions = model.predict(news_article_sequence)
predicted_class_index = np.argmax(predictions)
predicted_class_label = class_labels[predicted_class_index]
print("Predicted Class:", predicted_class_label)

1/1 [==============================] - 0s 26ms/step
Predicted Class: World


In [ ]:
import pickle
with open('tokenizerposneg.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model.save('/content/drive/MyDrive/whataboutMODEL/modelposneg.h5')
with open('/content/drive/MyDrive/whataboutMODEL/tokenizerposneg.pickle', 'wb') as file:
    pickle.dump(tokenizer, file)

Mounted at /content/drive
